In [4]:
import matplotlib.pyplot as plt
import numpy as np

from IPython.core.display import HTML
from itertools import chain
from collections import Counter, defaultdict
from helpers import show_model, Dataset
from pomegranate import State, HiddenMarkovModel, DiscreteDistribution

In [27]:
model = HiddenMarkovModel(name="tagger")
data = Dataset("tags-universal.txt", "brown-universal.txt", train_test_split=0.8)

# tag -> tag transitions needed for transition prob
def get_tags_pairs(tags_sequences):
    dual_counter = Counter()
    for tags in tags_sequences:
        for i in range(len(tags[:-1])):
            dual_counter[(tags[i], tags[i+1])] += 1
    return dual_counter

paired_tags = get_tags_pairs(data.training_set.Y)

# start/end -> tag transitions needed for transition prob
def get_start_end_tags(tags_sequences):
    start_tags = Counter()
    end_tags = Counter()
    for tags in tags_sequences:
        start_tags[tags[0]] += 1
        end_tags[tags[len(tags)-1]] += 1
    return start_tags, end_tags

start_tag, end_tag = get_start_end_tags(data.training_set.Y)

# tag to words mapping needed for emission propability
def get_tags_to_words_mapping(tags_sequence, words_sequence):
    word_tag_counter = defaultdict(Counter)
    for i in range(len(tags_sequence)):
        zipped_data = zip(tags_sequence[i], words_sequence[i])
        for tag, word in zipped_data:
            word_tag_counter[tag][word] += 1
    return word_tag_counter

emission_counts = get_tags_to_words_mapping(data.training_set.Y, data.training_set.X)

In [28]:
# adding emission propabilities ( from states to words )


model.add_states()

# adding transition propability ( between states )